<div style="text-align: center;">
    <h1>Registre national des entreprises (RNE)</h1>
</div>

Ce registre vient de [l'Institut National de la Propriété Industrielle](https://data.inpi.fr/) qui donne accès à de nombreuses données sur les entreprises.

Les données peuvent être consultées sur le web, sur le serveur de l'INPI ou par API. Il aurait été pratique d'utiliser l'API pour des questions de facilité de prise en main, de mise à jour des données etc ...

Néanmoins la [documentation officielle](https://www.inpi.fr/sites/default/files/documentation_technique_API_formalit%C3%A9s_v2.5.pdf) indique qu'il est impossible de filtrer géographiquement les établissements dans les méthodes GET et impose une limite de 10 Go / jour / utilisateur, ce qui est bloquant pour notre application.

On va donc utiliser les serveurs et télécharger les donnéees avec le protocole FTP.

Pour cela, il faut créer un compte INPI et faire la demande pour avoir les accès.

Une fois le compte crée, on peut accéder au serveur FTP de l'INPI (voir [ici](https://data.inpi.fr/content/editorial/Serveur_ftp_entreprises)).

Les données user_id / mdp sont retrouvables sur [ce lien](https://data.inpi.fr/espace_personnel/acces).

On télécharge le registre des Créations, modifications, cessations (CMC) d'entreprises.


Pour le télécharger, on peut s'inspirer du **code suivant** :

```bash
# connexion au serveur ftp de l'inpi
sftp user_id@www.inpi.net

# accepter la connexion et rentrer le mot de passe

# création d'un dossier sur la machine locale
lmkdir rne

# navigation dans ce dossier 
llcd rne

# téléchargement des fichiers
get stock RNE formalité.zip 

# le téléchargement dure 15 min environ

# décompression des données
unzip stock RNE formalité.zip 

In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm
import numpy as np
import pyarrow.parquet as pq

import sys

sys.path.append("../src/download")
from parse_rne import Entreprise, parse_formality

## 1.1 Exploration des données

A quoi ressemble le document ?

In [2]:
json_path = "../../rne/rne_deflated/stock_000001.json"


with open(json_path, "r") as file:
    text = json.load(file)

text[0]

{'updatedAt': '2023-03-06T16:48:28+01:00',
 'id': '64060b4c363a4f0cd40a5d33',
 'formality': {'siren': '001303346',
  'content': {'natureCreation': {'dateCreation': '2016-09-26',
    'societeEtrangere': False,
    'microEntreprise': False,
    'etablieEnFrance': True,
    'salarieEnFrance': True,
    'relieeEntrepriseAgricole': False,
    'entrepriseAgricole': True,
    'eirl': False},
   'personneMorale': {'identite': {'entreprise': {'siren': '001303346',
      'denomination': 'KATCHIT',
      'formeJuridique': '6901',
      'dateImmat': '2016-09-26T00:00:00+02:00'},
     'description': {'indicateurOrigineFusionScission': False}}}},
  'diffusionCommerciale': True},
 'siren': '001303346'}

Visiblement les données sont agglomérées au niveau du code siren (entreprise), pour chaque entreprise il y a une entrée par siret (établissement).

In [4]:
nb_établissements = len(text)
nb_fichiers = len(
    [file for file in os.listdir("../../rne/rne_deflated") if file.endswith("json")]
)

print(
    """Il y a {:,} entrées d'entreprises dans le fichier,
       soit environ {:,} entreprises au total sur les {:,} fichiers téléchargés""".format(
        nb_établissements, nb_fichiers * nb_établissements, nb_fichiers
    )
)

Il y a 99,997 entrées d'entreprises dans le fichier,
       soit environ 24,099,277 entreprises au total sur les 241 fichiers téléchargés


# 2. Tests

## 2.1 Sur une entreprise fictive

On a crée un dossier [test](../data/Données%20sites/test), modifiable à volonté et qui permet de tester différents scénarios.

En particulier on teste les cas suivants : 
- établissement principal fermé et les autres ouverts
- un établissement hors de France
- personne ni physique ni morale  (indivision)
- différents codes APE


In [5]:
file = "../data/test/test_parse_rne.json"

with open(file, "r") as file:
    text = json.load(file)

entreprise = Entreprise(text["formality"])

entreprise.etablissements_to_dict()

[{'codeApe': '17.21B',
  'siret': '06920075600034',
  'nomCommercial': 'CARTONNAGES MECHINAUD',
  'codeInseeCommune': '44150',
  'adresse': ' AV ST EXUPERY 44860 SAINT-AIGNAN-GRANDLIEU',
  'diffusionCommerciale': True}]

### 2.2 Sur un fichier json complet

In [25]:
%%time

df = pd.read_json(json_path)[["formality"]]

tqdm.pandas()

df["formality"] = df["formality"].progress_apply(lambda x: parse_formality(x))

df.dropna(subset=["formality"], inplace=True)

df = df.explode("formality")["formality"].apply(pd.Series)

100%|██████████| 99997/99997 [00:00<00:00, 187616.25it/s]


CPU times: user 4.42 s, sys: 5.52 s, total: 9.94 s
Wall time: 12.2 s


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11053 entries, 66 to 99907
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   codeApe               11053 non-null  object
 1   siret                 11053 non-null  object
 2   nomCommercial         11053 non-null  object
 3   codeInseeCommune      11053 non-null  object
 4   adresse               11053 non-null  object
 5   diffusionCommerciale  11053 non-null  bool  
dtypes: bool(1), object(5)
memory usage: 528.9+ KB


# 3. Sur tous les fichiers 

Parsing des fichiers

In [3]:
path = "../../rne/rne_deflated/"

files = [file for file in os.listdir(path) if file.endswith("json")]


dtypes = {
    "codeApe": "object",
    "siret": int,
    "nomCommercial": "object",
    "codeInseeCommune": "object",
    "adresse": "object",
    "diffusionCommerciale": "bool",
}

os.makedirs("../../rne/rne_parsed", exist_ok=True)

# parsing des fichiers json
for file in tqdm(files):
    df = pd.read_json(path + file)[["formality"]]
    df["formality"] = df["formality"].apply(lambda x: parse_formality(x))
    df.dropna(subset=["formality"], inplace=True)
    df = df.explode("formality")["formality"].apply(pd.Series)
    df.astype(dtypes, copy=False, errors="ignore")
    df.to_parquet(
        "../../rne/rne_parsed/" + file[:-5] + ".parquet",
        index=False,
        compression="gzip",
    )

  0%|          | 0/241 [00:00<?, ?it/s]

100%|██████████| 241/241 [53:12<00:00, 13.25s/it] 


Création d'un seul fichier

In [6]:
path = "../../rne/rne_parsed/"
files = [
    os.path.join(path, file) for file in os.listdir(path) if file.endswith("parquet")
]


schema = pq.ParquetFile(files[0]).schema_arrow
with pq.ParquetWriter(
    "../data/Données nationales/RNE.parquet", schema=schema
) as writer:
    for file in files:
        writer.write_table(pq.read_table(file, schema=schema))

# 4. Fichier final 

In [3]:
df_final = pd.read_parquet("../data/Données nationales/RNE.parquet")

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5074487 entries, 0 to 5074486
Data columns (total 6 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   codeApe               object
 1   siret                 object
 2   nomCommercial         object
 3   codeInseeCommune      object
 4   adresse               object
 5   diffusionCommerciale  bool  
dtypes: bool(1), object(5)
memory usage: 198.4+ MB


In [4]:
df_final.describe()

,codeApe,siret,nomCommercial,codeInseeCommune,adresse,diffusionCommerciale
count,5074487,5074487,5074487,5074487,5074487,5074487
unique,1378,5073991,1811798,34861,2895488,2
top,68.20B,,,75108,26 RUE DES RIGOLES 75020 PARIS 20,True
freq,476248,493,2286552,57927,2208,5074428


Certains siret ne sont pas uniques, c'est étrange pour un identifiant unique...
Quels sont les entreprises en cause ?

In [14]:
df_final.siret.value_counts().value_counts()

count
1      5073986
2            4
493          1
Name: count, dtype: int64

In [16]:
# get all duplicates of siret
df_final[df_final.siret.duplicated(keep=False)].sort_values(by="siret", ascending=False)

,codeApe,siret,nomCommercial,codeInseeCommune,adresse,diffusionCommerciale
2327277,.,95325826600014,SCI La Belle Vie,44036,33 RUE Michel Grimault 44110 Châteaubriant,True
2327279,.,95325826600014,SCI La Belle Vie,44036,33 RUE Michel Grimault 44110 Châteaubriant,True
3517279,56.10C,94890329900014,HAPPY SELENE,15187,6 AV Léon Bélard 15100 Saint-Flour,True
3517280,.,94890329900014,HAPPY SELENE,15014,1 RUE Sainte Anne 15000 Aurillac,True
3514948,46.39A,94832829900011,JBSAZ,97407,31 RUE François de Mahy 97420 Le Port,True
...,...,...,...,...,...,...
3514013,.,,SCI ROMANDRA WEISS,59042,797 RTE Nationale 59830 Bachy,True
3514008,.,,FOREVER PARIS,92035,113 AV du General Leclerc 92250 La Garenne-Col...,True
3514007,.,,SCI LE 36,62041,22 RUE de la Coignée 62000 Arras,True
3513995,.,,MB HOLDING,73008,21 BD maréchal de lattre de tassigny 73100 Aix...,True


Après vérification sur Pappers pour Happy selene [ici](https://www.pappers.fr/entreprise/happy-selene-948903299), les deux établissements ont le même siret.

Au vu du faible  nombre de cas on peut ne pas s'en inquiéter.

On va simplement changer le dtype

In [18]:
df_final["siret"] = df_final["siret"].fillna(0).replace("", 0).astype(int)

In [20]:
df_final.to_parquet(
    "../data/Données nationales/RNE.parquet", index=False, compression="gzip"
)